# Консервативность

Опишу ситуацию как я понял. Для начала надо создать дофига моделей линейных и пошуметь немного на них, и так померить консервативен ли тест или нет. Потом надо создать нелинейные модели и пошуметь уже на них, и там уже посмотреть на мощность критерия.

В первой части все выбирается из распределения $N(0,1)$. Как коэффициенты, так и переменные и шум, таким образом шум не слишком сильно перебивает данные.

In [437]:
from statsmodels.formula.api import ols
import scipy.stats as dst
import numpy as np
import pandas as pd

wt = []
ft = []
nobs = 100

for j in range(100):
    x = dst.norm(0,1).rvs((nobs, 4))                                               #"переменные"
    betas = dst.norm(0,1).rvs(4)*4                                                 #"коэффициенты"
    a = [x[i][0]*betas[0]+x[i][1]*betas[1]+
         x[i][2]*betas[2]+x[i][3]*betas[3]+
         dst.norm(0,1).rvs(1) for i in range(len(x))]                              #в реальности схема линейна + ШУМ
    a = np.reshape(a, (len(a), 1))
    yz = np.concatenate((x,a), 1)
    data = pd.DataFrame(yz, columns=['bib1', 'bib2', 'bib3', 'bib4', 'bib'])
    string = ""

    model4 = ols('bib ~ bib1 + bib2 + bib3 + bib4', data=data).fit()               #фитанул модель
  
    for s in data.columns[:-1]:                                                    #Итерируемся по гипотезам от 0 до 3
        string+=s+'='                                                              #Гипотезы "нулевая", "первая" и т.д.
        string+='0'
        wt.append(model4.wald_test(string, use_f=False).pvalue)
        ft.append(model4.f_test(string).pvalue)
        string = string[:-1]

Тут
вообще все модели линейные и мы должны всё отвергать, так как Фишер и Вальд проверяют на равенство нулю коэффициентов в линейной модели. Посчитаем количество гипотез, которые были отвергнуты (для того, чтобы посчитать консервативность) ($\alpha = 0.05$):

In [438]:
ind_w = [[] for i in range(4)]
for j in range(4):
    for i in range(int(len(wt)/4)):
        if wt[j+i*4] > 0.05:
            ind_w[j].append(i)
        
ind_f = [[] for i in range(4)]
for j in range(4):
    for i in range(int(len(ft)/4)):
        if ft[j+i*4] > 0.05:
            ind_f[j].append(i)

Тут, как видно, отрицаем мало, что и понятно, ведь изначально бралась именно линейная модель

In [440]:
print('те эксперименты, что были приняты тестом Фишера при "нулевой" гипотезе (bib1=0)')
print(ind_f[0])
print('те эксперименты, что были приняты тестом Фишера при "первой" гипотезе (bib1=bib2=0)')
print(ind_f[1])   
print('те эксперименты, что были приняты тестом Фишера при "второй" гипотезе (bib3=bib1=bib2=0) ')
print(ind_f[2])   
print('те эксперименты, что были приняты тестом Фишера при "третьей" гипотезе (bib4=bib3=bib1=bib2=0) ')
print(ind_f[3])

print('те эксперименты, что были приняты тестом Вальда при "нулевой" гипотезе (bib1=0)')
print(ind_w[0])
print('те эксперименты, что были приняты тестом Вальда при "первой" гипотезе (bib1=bib2=0)')
print(ind_w[1])   
print('те эксперименты, что были приняты тестом Вальда при "второй" гипотезе (bib3=bib1=bib2=0) ')
print(ind_w[2])   
print('те эксперименты, что были приняты тестом Вальда при "третьей" гипотезе (bib4=bib3=bib1=bib2=0) ')
print(ind_w[3])

те эксперименты, что были приняты тестом Фишера при "нулевой" гипотезе (bib1=0)
[4]
те эксперименты, что были приняты тестом Фишера при "первой" гипотезе (bib1=bib2=0)
[]
те эксперименты, что были приняты тестом Фишера при "второй" гипотезе (bib3=bib1=bib2=0) 
[]
те эксперименты, что были приняты тестом Фишера при "третьей" гипотезе (bib4=bib3=bib1=bib2=0) 
[]
те эксперименты, что были приняты тестом Вальда при "нулевой" гипотезе (bib1=0)
[4]
те эксперименты, что были приняты тестом Вальда при "первой" гипотезе (bib1=bib2=0)
[]
те эксперименты, что были приняты тестом Вальда при "второй" гипотезе (bib3=bib1=bib2=0) 
[]
те эксперименты, что были приняты тестом Вальда при "третьей" гипотезе (bib4=bib3=bib1=bib2=0) 
[]


Далее будем смотреть на мощность и на консервативность выборки. Я точно так и не понял из материала презентаций какую выбирать нулевую гипотезу, поэтому сделал их по 4 (зануляя 1, 1 и 2, 1 и 2 и 3, и все коэффициенты). Во избежание каких либо недопониманий, насколько мне известно, Никита написал тебе и ты ему в ответ прислал ссылку https://en.wikipedia.org/wiki/Statistical_hypothesis_testing, чем я и буду руководствоваться. Тогда пусть $nominal significane level = 0.05$  Для них и посчитаю следующие значения:

In [441]:
print('Консервативен ли тест Вальда для "нулевой" гипотезы?', len(ind_w[0])/100 < 0.05)
print('Консервативен ли тест Вальда для "первой" гипотезы?', len(ind_w[1])/100 < 0.05)
print('Консервативен ли тест Вальда для "второй" гипотезы?', len(ind_w[2])/100 < 0.05)
print('Консервативен ли тест Вальда для "третьей" гипотезы?', len(ind_w[3])/100 < 0.05)

print('Консервативен ли тест Фишера для "нулевой" гипотезы?', len(ind_f[0])/100 < 0.05)
print('Консервативен ли тест Фишера для "первой" гипотезы?', len(ind_f[1])/100 < 0.05)
print('Консервативен ли тест Фишера для "второй" гипотезы?', len(ind_f[2])/100 < 0.05)
print('Консервативен ли тест Фишера для "третьей" гипотезы?', len(ind_f[3])/100 < 0.05)

Консервативен ли тест Вальда для "нулевой" гипотезы? True
Консервативен ли тест Вальда для "первой" гипотезы? True
Консервативен ли тест Вальда для "второй" гипотезы? True
Консервативен ли тест Вальда для "третьей" гипотезы? True
Консервативен ли тест Фишера для "нулевой" гипотезы? True
Консервативен ли тест Фишера для "первой" гипотезы? True
Консервативен ли тест Фишера для "второй" гипотезы? True
Консервативен ли тест Фишера для "третьей" гипотезы? True


Оценим истинную вероятность отвержения нулевой гипотезы при её истинности с помощью биномиального теста

In [442]:
print('"Нулевая" гипотеза для теста Фишера, консервативность: ', st.binom_test(len(ind_f[0]), 100, p = 0.05, alternative = 'greater'))
print('"Первая" гипотеза для теста Фишера, консервативность: ', st.binom_test(len(ind_f[1]), 100, p = 0.05, alternative = 'greater'))
print('"Вторая" гипотеза для теста Фишера, консервативность: ', st.binom_test(len(ind_f[2]), 100, p = 0.05, alternative = 'greater'))
print('"Третья" гипотеза для теста Фишера, консервативность: ', st.binom_test(len(ind_f[3]), 100, p = 0.05, alternative = 'greater'))

print('"Нулевая" гипотеза для теста Вальда, консервативность: ', st.binom_test(len(ind_w[0]), 100, p = 0.05, alternative = 'greater'))
print('"Первая" гипотеза для теста Вальда, консервативность: ', st.binom_test(len(ind_w[1]), 100, p = 0.05, alternative = 'greater'))
print('"Вторая" гипотеза для теста Вальда, консервативность: ', st.binom_test(len(ind_w[2]), 100, p = 0.05, alternative = 'greater'))
print('"Третья" гипотеза для теста Вальда, консервативность: ', st.binom_test(len(ind_w[3]), 100, p = 0.05, alternative = 'greater'))

"Нулевая" гипотеза для теста Фишера, консервативность:  0.994079470779666
"Первая" гипотеза для теста Фишера, консервативность:  1.0
"Вторая" гипотеза для теста Фишера, консервативность:  1.0
"Третья" гипотеза для теста Фишера, консервативность:  1.0
"Нулевая" гипотеза для теста Вальда, консервативность:  0.994079470779666
"Первая" гипотеза для теста Вальда, консервативность:  1.0
"Вторая" гипотеза для теста Вальда, консервативность:  1.0
"Третья" гипотеза для теста Вальда, консервативность:  1.0


Тут вбирал всё из одинаковых $\chi^2$ выборок. 

In [443]:
wt1 = []
ft1 = []

for j in range(100):
    x = dst.chi2(2).rvs((nobs, 4))
    betas = dst.chi2(2).rvs(4)*4
    a = [x[i][0]*betas[0]+x[i][1]*betas[1]+
         x[i][2]*betas[2]+x[i][3]*betas[3]+
         dst.chi2(2).rvs(1) for i in range(len(x))]
    a = np.reshape(a, (len(a), 1))
    yz = np.concatenate((x,a), 1)
    data = pd.DataFrame(yz, columns=['bib1', 'bib2', 'bib3', 'bib4', 'bib'])
    string = ""

    model4 = ols('bib ~ bib1 + bib2 + bib3 + bib4', data=data).fit()


    for s in data.columns[:-1]:
        string+=s+'='
        string+='0'
        wt1.append(model4.wald_test(string, use_f=False).pvalue)
        ft1.append(model4.f_test(string).pvalue)
        string = string[:-1]


In [444]:
ind_w = [[] for i in range(4)]
for j in range(4):
    for i in range(int(len(wt1)/4)):
        if wt1[j+i*4] > 0.05:
            ind_w[j].append(i)
        
ind_f = [[] for i in range(4)]
for j in range(4):
    for i in range(int(len(ft1)/4)):
        if ft1[j+i*4] > 0.05:
            ind_f[j].append(i)

In [445]:
print('те эксперименты, что были приняты тестом Фишера при "нулевой" гипотезе (bib1=0)')
print(ind_f[0])
print('те эксперименты, что были приняты тестом Фишера при "первой" гипотезе (bib1=bib2=0)')
print(ind_f[1])   
print('те эксперименты, что были приняты тестом Фишера при "второй" гипотезе (bib3=bib1=bib2=0) ')
print(ind_f[2])   
print('те эксперименты, что были приняты тестом Фишера при "третьей" гипотезе (bib4=bib3=bib1=bib2=0) ')
print(ind_f[3])

print('те эксперименты, что были приняты тестом Вальда при "нулевой" гипотезе (bib1=0)')
print(ind_w[0])
print('те эксперименты, что были приняты тестом Вальда при "первой" гипотезе (bib1=bib2=0)')
print(ind_w[1])   
print(' эксперименты, что были приняты тестом Вальда при "второй" гипотезе (bib3=bib1=bib2=0) ')
print(ind_w[2])   
print(' те эксперименты, что были приняты тестом Вальда при "третьей" гипотезе (bib4=bib3=bib1=bib2=0) ')
print(ind_w[3])

те эксперименты, что были приняты тестом Фишера при "нулевой" гипотезе (bib1=0)
[70, 81]
те эксперименты, что были приняты тестом Фишера при "первой" гипотезе (bib1=bib2=0)
[]
те эксперименты, что были приняты тестом Фишера при "второй" гипотезе (bib3=bib1=bib2=0) 
[]
те эксперименты, что были приняты тестом Фишера при "третьей" гипотезе (bib4=bib3=bib1=bib2=0) 
[]
те эксперименты, что были приняты тестом Вальда при "нулевой" гипотезе (bib1=0)
[70, 81]
те эксперименты, что были приняты тестом Вальда при "первой" гипотезе (bib1=bib2=0)
[]
 эксперименты, что были приняты тестом Вальда при "второй" гипотезе (bib3=bib1=bib2=0) 
[]
 те эксперименты, что были приняты тестом Вальда при "третьей" гипотезе (bib4=bib3=bib1=bib2=0) 
[]


In [446]:
print('Консервативен ли тест Вальда для "нулевой гипотезы?', len(ind_w[0])/100 < 0.05)
print('Консервативен ли тест Вальда для "первой" гипотезы?', len(ind_w[1])/100 < 0.05)
print('Консервативен ли тест Вальда для "второй" гипотезы?', len(ind_w[2])/100 < 0.05)
print('Консервативен ли тест Вальда для "третьей" гипотезы?', len(ind_w[3])/100 < 0.05)

print('Консервативен ли тест Фишера для "нулевой" гипотезы?', len(ind_f[0])/100 < 0.05)
print('Консервативен ли тест Фишера для "первой" гипотезы?', len(ind_f[1])/100 < 0.05)
print('Консервативен ли тест Фишера для "второй" гипотезы?', len(ind_f[2])/100 < 0.05)
print('Консервативен ли тест Фишера для "третьей" гипотезы?', len(ind_f[3])/100 < 0.05)

Консервативен ли тест Вальда для "нулевой гипотезы? True
Консервативен ли тест Вальда для "первой" гипотезы? True
Консервативен ли тест Вальда для "второй" гипотезы? True
Консервативен ли тест Вальда для "третьей" гипотезы? True
Консервативен ли тест Фишера для "нулевой" гипотезы? True
Консервативен ли тест Фишера для "первой" гипотезы? True
Консервативен ли тест Фишера для "второй" гипотезы? True
Консервативен ли тест Фишера для "третьей" гипотезы? True


In [447]:
print('"Нулевая" гипотеза для теста Фишера, консервативность: ', st.binom_test(len(ind_f[0]), 100, p = 0.05, alternative = 'greater'))
print('"Первая" гипотеза для теста Фишера, консервативность: ', st.binom_test(len(ind_f[1]), 100, p = 0.05, alternative = 'greater'))
print('"Вторая" гипотеза для теста Фишера, консервативность: ', st.binom_test(len(ind_f[2]), 100, p = 0.05, alternative = 'greater'))
print('"Третья" гипотеза для теста Фишера, консервативность: ', st.binom_test(len(ind_f[3]), 100, p = 0.05, alternative = 'greater'))

print('"Нулевая" гипотеза для теста Вальда, консервативность: ', st.binom_test(len(ind_w[0]), 100, p = 0.05, alternative = 'greater'))
print('"Первая" гипотеза для теста Вальда, консервативность: ', st.binom_test(len(ind_w[1]), 100, p = 0.05, alternative = 'greater'))
print('"Вторая" гипотеза для теста Вальда, консервативность: ', st.binom_test(len(ind_w[2]), 100, p = 0.05, alternative = 'greater'))
print('"Третья" гипотеза для теста Вальда, консервативность: ', st.binom_test(len(ind_w[3]), 100, p = 0.05, alternative = 'greater'))

"Нулевая" гипотеза для теста Фишера, консервативность:  0.9629187906726451
"Первая" гипотеза для теста Фишера, консервативность:  1.0
"Вторая" гипотеза для теста Фишера, консервативность:  1.0
"Третья" гипотеза для теста Фишера, консервативность:  1.0
"Нулевая" гипотеза для теста Вальда, консервативность:  0.9629187906726451
"Первая" гипотеза для теста Вальда, консервативность:  1.0
"Вторая" гипотеза для теста Вальда, консервативность:  1.0
"Третья" гипотеза для теста Вальда, консервативность:  1.0


In [448]:
wt2 = []
ft2 = []

for j in range(100):
    x = dst.t(2).rvs((nobs, 4))
    betas = dst.t(2).rvs(4)*4
    a = [x[i][0]*betas[0]+x[i][1]*betas[1]+
         x[i][2]*betas[2]+x[i][3]*betas[3]+
         dst.t(2).rvs(1) for i in range(len(x))]
    a = np.reshape(a, (len(a), 1))
    yz = np.concatenate((x,a), 1)
    data = pd.DataFrame(yz, columns=['bib1', 'bib2', 'bib3', 'bib4', 'bib'])
    string = ""

    model4 = ols('bib ~ bib1 + bib2 + bib3 + bib4', data=data).fit()


    for s in data.columns[:-1]:
        string+=s+'='
        string+='0'
        wt2.append(model4.wald_test(string, use_f=False).pvalue)
        ft2.append(model4.f_test(string).pvalue)
        string = string[:-1]

In [449]:
ind_w = [[] for i in range(4)]
for j in range(4):
    for i in range(int(len(wt2)/4)):
        if wt2[j+i*4] > 0.05:
            ind_w[j].append(i)
        
ind_f = [[] for i in range(4)]
for j in range(4):
    for i in range(int(len(ft2)/4)):
        if ft2[j+i*4] > 0.05:
            ind_f[j].append(i)

In [453]:
print('те эксперименты, что были приняты тестом Фишера при "нулевой гипотезе (bib1=0)')
print(ind_f[0])
print('те эксперименты, что были приняты тестом Фишера при "первой гипотезе (bib1=bib2=0)')
print(ind_f[1])   
print('те эксперименты, что были приняты тестом Фишера при "второй гипотезе (bib3=bib1=bib2=0) ')
print(ind_f[2])   
print('те эксперименты, что были приняты тестом Фишера при "третьей гипотезе (bib4=bib3=bib1=bib2=0) ')
print(ind_f[3])

print('те эксперименты, что были приняты тестом Вальда при "нулевой" гипотезе (bib1=0)')
print(ind_w[0])
print('те эксперименты, что были приняты тестом Вальда при "первой" гипотезе (bib1=bib2=0)')
print(ind_w[1])   
print('те эксперименты, что были приняты тестом Вальда при "второй" гипотезе (bib3=bib1=bib2=0) ')
print(ind_w[2])   
print('те эксперименты, что были приняты тестом Вальда при "третьей" гипотезе (bib4=bib3=bib1=bib2=0) ')
print(ind_w[3])

те эксперименты, что были приняты тестом Фишера при "нулевой гипотезе (bib1=0)
[8, 12, 37, 49, 52, 55, 63]
те эксперименты, что были приняты тестом Фишера при "первой гипотезе (bib1=bib2=0)
[]
те эксперименты, что были приняты тестом Фишера при "второй гипотезе (bib3=bib1=bib2=0) 
[]
те эксперименты, что были приняты тестом Фишера при "третьей гипотезе (bib4=bib3=bib1=bib2=0) 
[]
те эксперименты, что были приняты тестом Вальда при "нулевой" гипотезе (bib1=0)
[8, 12, 37, 49, 52, 55, 63]
те эксперименты, что были приняты тестом Вальда при "первой" гипотезе (bib1=bib2=0)
[]
те эксперименты, что были приняты тестом Вальда при "второй" гипотезе (bib3=bib1=bib2=0) 
[]
те эксперименты, что были приняты тестом Вальда при "третьей" гипотезе (bib4=bib3=bib1=bib2=0) 
[]


In [454]:
print('Консервативен ли тест Вальда для "нулевой" гипотезы?', len(ind_w[0])/100 < 0.05)
print('Консервативен ли тест Вальда для "первой" гипотезы?', len(ind_w[1])/100 < 0.05)
print('Консервативен ли тест Вальда для "второй" гипотезы?', len(ind_w[2])/100 < 0.05)
print('Консервативен ли тест Вальда для "третьей" гипотезы?', len(ind_w[3])/100 < 0.05)

print('Консервативен ли тест Фишера для "нулевой" гипотезы?', len(ind_f[0])/100 < 0.05)
print('Консервативен ли тест Фишера для "первой" гипотезы?', len(ind_f[1])/100 < 0.05)
print('Консервативен ли тест Фишера для "второй" гипотезы?', len(ind_f[2])/100 < 0.05)
print('Консервативен ли тест Фишера для "третьей" гипотезы?', len(ind_f[3])/100 < 0.05)

Консервативен ли тест Вальда для "нулевой" гипотезы? False
Консервативен ли тест Вальда для "первой" гипотезы? True
Консервативен ли тест Вальда для "второй" гипотезы? True
Консервативен ли тест Вальда для "третьей" гипотезы? True
Консервативен ли тест Фишера для "нулевой" гипотезы? False
Консервативен ли тест Фишера для "первой" гипотезы? True
Консервативен ли тест Фишера для "второй" гипотезы? True
Консервативен ли тест Фишера для "третьей" гипотезы? True


In [455]:
print('"Нулевая" гипотеза для теста Фишера, консервативность: ', st.binom_test(len(ind_f[0]), 100, p = 0.05, alternative = 'greater'))
print('"Первая" гипотеза для теста Фишера, консервативность: ', st.binom_test(len(ind_f[1]), 100, p = 0.05, alternative = 'greater'))
print('"Вторая" гипотеза для теста Фишера, консервативность: ', st.binom_test(len(ind_f[2]), 100, p = 0.05, alternative = 'greater'))
print('"Третья" гипотеза для теста Фишера, консервативность: ', st.binom_test(len(ind_f[3]), 100, p = 0.05, alternative = 'greater'))

print('"Нулевая" гипотеза для теста Вальда, консервативность: ', st.binom_test(len(ind_w[0]), 100, p = 0.05, alternative = 'greater'))
print('"Первая" гипотеза для теста Вальда, консервативность: ', st.binom_test(len(ind_w[1]), 100, p = 0.05, alternative = 'greater'))
print('"Вторая" гипотеза для теста Вальда, консервативность: ', st.binom_test(len(ind_w[2]), 100, p = 0.05, alternative = 'greater'))
print('"Третья" гипотеза для теста Вальда, консервативность: ', st.binom_test(len(ind_w[3]), 100, p = 0.05, alternative = 'greater'))

"Нулевая" гипотеза для теста Фишера, консервативность:  0.23398601598516702
"Первая" гипотеза для теста Фишера, консервативность:  1.0
"Вторая" гипотеза для теста Фишера, консервативность:  1.0
"Третья" гипотеза для теста Фишера, консервативность:  1.0
"Нулевая" гипотеза для теста Вальда, консервативность:  0.23398601598516702
"Первая" гипотеза для теста Вальда, консервативность:  1.0
"Вторая" гипотеза для теста Вальда, консервативность:  1.0
"Третья" гипотеза для теста Вальда, консервативность:  1.0


# Мощность

In [456]:
wt3 = []
ft3 = []

for j in range(100):
    x = np.random.randn(nobs, 4)                                                  #"переменные"
    betas = np.random.randn(4)*4                                                  #"коэффициенты"
    a = [x[i][0]*betas[0]+x[i][1]*betas[1]+
         x[i][2]*betas[2]*x[i][2]*betas[2]+x[i][3]*betas[3]+
         dst.norm(0, 1).rvs(1)for i in range(len(x))]                             #в реальности схема квадратична + ШУМ
    a = np.reshape(a, (len(a), 1))
    data = np.concatenate((x,a), 1)
    data = pd.DataFrame(data, columns=['bib1', 'bib2', 'bib3', 'bib4', 'bib'])    
    string = ""

    model4 = ols('bib ~ bib1 + bib2 + bib3 + bib4', data=data).fit()              #фитанул модель

    for s in data.columns[:-1]:                                                   #Итерируемся по гипотезам от 0 до 3
        string+=s+'='
        string+='0'
        wt3.append(model4.wald_test(string, use_f=False).pvalue)
        ft3.append(model4.f_test(string).pvalue)
        string = string[:-1]

In [460]:
ind_w = [[] for i in range(4)]
for j in range(4):
    for i in range(int(len(wt3)/4)):
        if wt3[j+i*4] < 0.05:
            ind_w[j].append(i)
        
ind_f = [[] for i in range(4)]
for j in range(4):
    for i in range(int(len(ft3)/4)):
        if ft3[j+i*4] < 0.05:
            ind_f[j].append(i)

In [461]:
print('те эксперименты, что были отвергнуты тестом Фишера при "нулевой" гипотезе (bib1=0)')
print(ind_f[0])
print('те эксперименты, что были отвергнуты тестом Фишера при "первой" гипотезе (bib1=bib2=0)')
print(ind_f[1])   
print('те эксперименты, что были отвергнуты тестом Фишера при "второй" гипотезе (bib3=bib1=bib2=0) ')
print(ind_f[2])   
print('те эксперименты, что были отвергнуты тестом Фишера при "третьей" гипотезе (bib4=bib3=bib1=bib2=0) ')
print(ind_f[3])

print('те эксперименты, что были отвергнуты тестом Вальда при "нулевой" гипотезе (bib1=0)')
print(ind_w[0])
print('те эксперименты, что были отвергнуты тестом Вальда при "первой" гипотезе (bib1=bib2=0)')
print(ind_w[1])   
print('те эксперименты, что были отвергнуты тестом Вальда при "второй" гипотезе (bib3=bib1=bib2=0) ')
print(ind_w[2])   
print('те эксперименты, что были отвергнуты тестом Вальда при "третьей" гипотезе (bib4=bib3=bib1=bib2=0) ')
print(ind_w[3])

те эксперименты, что были отвергнуты тестом Фишера при "нулевой" гипотезе (bib1=0)
[0, 1, 2, 3, 4, 5, 7, 8, 10, 12, 15, 18, 19, 21, 22, 23, 27, 28, 29, 30, 31, 34, 36, 37, 38, 40, 41, 45, 48, 49, 50, 51, 52, 53, 54, 55, 58, 59, 60, 62, 63, 64, 65, 67, 69, 70, 71, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 91, 92, 93, 96, 97, 99]
те эксперименты, что были отвергнуты тестом Фишера при "первой" гипотезе (bib1=bib2=0)
[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 12, 15, 17, 18, 19, 20, 21, 22, 23, 28, 29, 30, 31, 34, 36, 37, 38, 40, 41, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 75, 76, 77, 79, 80, 81, 82, 83, 84, 85, 87, 88, 91, 92, 93, 96, 97, 98, 99]
те эксперименты, что были отвергнуты тестом Фишера при "второй" гипотезе (bib3=bib1=bib2=0) 
[0, 1, 2, 3, 4, 5, 7, 8, 10, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 27, 28, 29, 30, 31, 32, 33, 34, 37, 38, 40, 41, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,

Далее считаем мощность. (По идее тут должно было всё отвергаться, но так не происходило):

In [462]:
print('Мощность критерия Вальда на "нулевой" гипотезе', len(ind_f[0])/100)
print('Мощность критерия Вальда на "первой" гипотезе', len(ind_f[1])/100)
print('Мощность критерия Вальда на "второй" гипотезе', len(ind_f[2])/100)
print('Мощность критерия Вальда на "третьей" гипотезе', len(ind_f[3])/100)

print('Мощность критерия Вальда на "нулевой" гипотезе', len(ind_w[0])/100)
print('Мощность критерия Вальда на "первой" гипотезе', len(ind_w[1])/100)
print('Мощность критерия Вальда на "второй" гипотезе', len(ind_w[2])/100)
print('Мощность критерия Вальда на "третьей" гипотезе', len(ind_w[3])/100)


Мощность критерия Вальда на "нулевой" гипотезе 0.66
Мощность критерия Вальда на "первой" гипотезе 0.71
Мощность критерия Вальда на "второй" гипотезе 0.78
Мощность критерия Вальда на "третьей" гипотезе 0.84
Мощность критерия Вальда на "нулевой" гипотезе 0.67
Мощность критерия Вальда на "первой" гипотезе 0.71
Мощность критерия Вальда на "второй" гипотезе 0.8
Мощность критерия Вальда на "третьей" гипотезе 0.84


In [463]:
wt4 = []
ft4 = []

for j in range(100):
    x = np.random.randn(nobs, 4)                                                  #"переменные"
    betas = np.random.randn(4)*4                                                  #"коэффициенты"
    a = [x[i][0]*betas[0]+x[i][1]*betas[1]+
         x[i][2]*betas[2]*x[i][2]*betas[2]*x[i][2]*betas[2]+
         x[i][3]*betas[3]*x[i][3]*betas[3]+
         dst.norm(0, 1).rvs(1)for i in range(len(x))]                             #в реальности схема кубическая + ШУМ
    a = np.reshape(a, (len(a), 1))
    data = np.concatenate((x,a), 1)
    data = pd.DataFrame(data, columns=['bib1', 'bib2', 'bib3', 'bib4', 'bib'])    
    string = ""

    model4 = ols('bib ~ bib1 + bib2 + bib3 + bib4', data=data).fit()              #фитанул модель

    for s in data.columns[:-1]:                                                   #Итерируемся по гипотезам от 0 до 3
        string+=s+'='
        string+='0'
        wt4.append(model4.wald_test(string, use_f=False).pvalue)
        ft4.append(model4.f_test(string).pvalue)
        string = string[:-1]

In [464]:
ind_w = [[] for i in range(4)]
for j in range(4):
    for i in range(int(len(wt4)/4)):
        if wt4[j+i*4] < 0.05:
            ind_w[j].append(i)
        
ind_f = [[] for i in range(4)]
for j in range(4):
    for i in range(int(len(ft4)/4)):
        if ft4[j+i*4] < 0.05:
            ind_f[j].append(i)

In [465]:
print('те эксперименты, что были отвергнуты тестом Фишера при "нулевой" гипотезе (bib1=0)')
print(ind_f[0])
print('те эксперименты, что были отвергнуты тестом Фишера при "первой" гипотезе (bib1=bib2=0)')
print(ind_f[1])   
print('те эксперименты, что были отвергнуты тестом Фишера при "второй" гипотезе (bib3=bib1=bib2=0) ')
print(ind_f[2])   
print('те эксперименты, что были отвергнуты тестом Фишера при "третьей" гипотезе (bib4=bib3=bib1=bib2=0) ')
print(ind_f[3])

print('те эксперименты, что были отвергнуты тестом Вальда при "нулевой" гипотезе (bib1=0)')
print(ind_w[0])
print('те эксперименты, что были отвергнуты тестом Вальда при "первой" гипотезе (bib1=bib2=0)')
print(ind_w[1])   
print('те эксперименты, что были отвергнуты тестом Вальда при "второй" гипотезе (bib3=bib1=bib2=0) ')
print(ind_w[2])   
print('те эксперименты, что были отвергнуты тестом Вальда при "третьей" гипотезе (bib4=bib3=bib1=bib2=0) ')
print(ind_w[3])

те эксперименты, что были отвергнуты тестом Фишера при "нулевой" гипотезе (bib1=0)
[2, 3, 7, 8, 13, 14, 16, 28, 30, 31, 35, 42, 48, 52, 56, 57, 58, 63, 68, 85, 87, 92, 93, 98]
те эксперименты, что были отвергнуты тестом Фишера при "первой" гипотезе (bib1=bib2=0)
[2, 3, 7, 8, 13, 14, 16, 20, 21, 28, 30, 33, 48, 52, 56, 57, 63, 68, 73, 74, 79, 81, 85, 86, 92, 93, 97, 98]
те эксперименты, что были отвергнуты тестом Фишера при "второй" гипотезе (bib3=bib1=bib2=0) 
[0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 60, 61, 62, 63, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 98, 99]
те эксперименты, что были отвергнуты тестом Фишера при "третьей" гипотезе (bib4=bib3=bib1=bib2=0) 
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,

Далее считаем мощность. (По идее тут должно было всё отрицаться, но так не происходило):

In [466]:
print('Мощность критерия Вальда на "нулевой" гипотезе', len(ind_f[0])/100)
print('Мощность критерия Вальда на "первой" гипотезе', len(ind_f[1])/100)
print('Мощность критерия Вальда на "второй" гипотезе', len(ind_f[2])/100)
print('Мощность критерия Вальда на "третьей" гипотезе', len(ind_f[3])/100)

print('Мощность критерия Вальда на "нулевой" гипотезе', len(ind_w[0])/100)
print('Мощность критерия Вальда на "первой" гипотезе', len(ind_w[1])/100)
print('Мощность критерия Вальда на "второй" гипотезе', len(ind_w[2])/100)
print('Мощность критерия Вальда на "третьей" гипотезе', len(ind_w[3])/100)


Мощность критерия Вальда на "нулевой" гипотезе 0.24
Мощность критерия Вальда на "первой" гипотезе 0.28
Мощность критерия Вальда на "второй" гипотезе 0.89
Мощность критерия Вальда на "третьей" гипотезе 0.92
Мощность критерия Вальда на "нулевой" гипотезе 0.25
Мощность критерия Вальда на "первой" гипотезе 0.29
Мощность критерия Вальда на "второй" гипотезе 0.89
Мощность критерия Вальда на "третьей" гипотезе 0.93


# Выводы

Как можно заметить из проведённых экспериментов, критерии Вальда и Фишера в большинстве случаев консервативны (как при ручной проверке вероятностями, так и при биномиальном тесте), а мощность в целом имеет большие амплитуды и величина данной амплитуды сильно зависит от того, насколько сильно мы "отдалили" модель от линейной (в первой проверке на мощность была квадратичная зависимость и там варьировалось от 0,66 до 0,84, а во второй проверке с кубической зависимостью от 0,24 до 0,92 в зависимости от нулевой гипотезы)